# Imports

In [162]:
import pandas as pd
import numpy as np

In [163]:
# Importation des csv 
df_vf = pd.read_csv("../csv_clean/valeurs_foncieres_2024_clean_values.csv")
df_vf_old = pd.read_csv("../csv_clean/valeurs_foncieres_2020_clean_values.csv")
df_parcs = pd.read_csv("../csv_clean/parcs_jardins_communes_agg.csv")
df_college = pd.read_csv("../csv_clean/college_clean.csv")
df_lycee = pd.read_csv("../csv_clean/lycee_clean.csv")

# Tries besoins

In [164]:
# besoins 
nbr_piece = 4 # minimum 
rapport_qualite_prix = 0.75 #% minimum 
investissement = 0.5

# qualité 
surface = 70 # minimum 
score_vert = 75 #% minimum 
ecole = 5 # minimum

In [165]:
dict_24 = df_vf.groupby("nom_commune")["valeur_fonciere"].mean().to_dict()
dict_20 = df_vf_old.groupby("nom_commune")["valeur_fonciere"].mean().to_dict()

In [166]:
commune_names = []
bad_inv_name = []

for key_24, key_20 in zip(dict_24, dict_20):
    if dict_24[key_24] / dict_20[key_20] < investissement:
        bad_inv_name.append(key_24)
    else:
        commune_names.append(key_24)

print(bad_inv_name)
print(len(bad_inv_name), '/', len(dict_24), '\n')

['arnas', 'azolette', 'blacé', 'caluire-et-cuire', 'cercié', 'chasselay', 'chaussan', 'chénas', 'dardilly', 'juliénas', 'lantignié', 'longessaigne', 'lucenay', 'lyon 1er arrondissement', 'lyon 5e arrondissement', 'lyon 7e arrondissement', 'marchampt', 'meaux-la-montagne', 'meys', 'moiré', 'montrottier', 'odenas', 'pomeys', 'propières', 'ranchal', 'rivolet', 'rontalon', 'saint-appolinaire', 'saint-bonnet-des-bruyères', 'saint-bonnet-le-troncy', 'saint-clément-sur-valsonne', 'saint-cyr-le-chatoux', 'saint-didier-sur-beaujeu', "saint-genis-l'argentière", 'saint-igny-de-vers', "saint-just-d'avray", 'saint-laurent-de-chamousset', "saint-nizier-d'azergues", 'saint-pierre-la-palud', 'saint-romain-en-gier', 'saint-vincent-de-reins', 'sainte-colombe', "sainte-foy-l'argentière", 'sarcey', 'souzy', 'taponas', 'thizy-les-bourgs', 'valsonne', 'vauxrenard', 'vernay', 'villechenève', 'villefranche-sur-saône', 'yzeron', 'échalas', 'émeringes']
55 / 276 



In [167]:
def filtred_df(df):
    # join (concaténations) des collèges et lycées 
    df_ecoles = pd.concat([df_college, df_lycee])
    df_ecoles["nb_ecoles"] = df_ecoles.groupby("commune")["commune"].transform("count")

    # join (merge) les valeurs foncières et les parcs
    df = df.merge(df_parcs[["code_commune", "score_espaces_verts"]], on="code_commune", how="left")

    # join (merge) de dataframe et les écoles 
    df = df.merge(df_ecoles[["code_postal", "nb_ecoles"]], on="code_postal", how="left")

    # sup les communes à mauvais investissement
    df = df[~df["nom_commune"].isin(bad_inv_name)]

    # call function to filter with room and quality/price
    df = filtered_room_quality_rpice(df)
    
    return df

In [168]:
def filtered_room_quality_rpice(df):
    # définir qualité pour faire rapport qualité/prix
    df["qualite"] = np.where(
        (df["surface_reelle_bati"] >= surface) & (df["score_espaces_verts"] >= score_vert) & (df["nb_ecoles"] >= ecole),
        1,  # très bonne qualité
        np.where(
            (df["surface_reelle_bati"] >= surface) | (df["score_espaces_verts"] >= score_vert) | (df["nb_ecoles"] >= ecole),
            0.5,  # qualité moyenne
            0   # faible qualité
        )
    )
    print('Qualité moyenne :', df["qualite"].mean())

    # calcul du prix moyen dans la région lyonnaise 
    prix_moyen_lyon = df["valeur_fonciere"].mean()
    print('Prix moyen de Lyon :', round(prix_moyen_lyon))

    # qualité / prix 
    df["qualite_prix"] = (df["qualite"] / df["valeur_fonciere"]) * prix_moyen_lyon
    print('Rapport qualité prix moyen :', df["qualite_prix"].mean())

    print('Nombre pièces moyennes :', df["nombre_pieces_principales"].mean())
    print('Nombre pièces max :', df["nombre_pieces_principales"].max())

    # filtrage des données 
    filtred_df = df[
        (df["nombre_pieces_principales"] >= nbr_piece) &
        (df["qualite_prix"] >= rapport_qualite_prix)
    ].copy()

    print(filtred_df.shape)
    return filtred_df

In [169]:
df24 = filtred_df(df_vf)
print('\n', df24["nom_commune"].unique())

Qualité moyenne : 0.4536165015605917
Prix moyen de Lyon : 570594
Rapport qualité prix moyen : 2132.479779926978
Nombre pièces moyennes : 1.2837639510962764
Nombre pièces max : 28.0
(29181, 14)

 ['lyon 4e arrondissement' 'oullins' 'loire-sur-rhône' 'rillieux-la-pape'
 'bessenay' 'fontaines-sur-saône' 'beauvallon' 'millery'
 'saint-germain-nuelles' 'lyon 9e arrondissement' 'quincieux'
 "champagne-au-mont-d'or" 'savigny' 'francheville' 'cublize'
 'vindry-sur-turdine' 'porte des pierres dorées' 'gleizé'
 'tassin-la-demi-lune' 'brignais' 'belleville-en-beaujolais' 'anse'
 'sainte-foy-lès-lyon' 'vernaison' 'saint-jean-la-bussière' 'lozanne'
 'chamelet' 'écully' 'mornant' 'condrieu' 'irigny' 'cours'
 'lyon 2e arrondissement' 'rochetaillée-sur-saône' 'limas' 'grandris'
 'tupin-et-semons' 'bully' 'saint-genis-laval' 'quincié-en-beaujolais'
 'saint-romain-en-gal' 'lentilly' 'craponne' 'genay'
 "saint-marcel-l'éclairé" 'poule-les-écharmeaux'
 'saint-georges-de-reneins' 'brindas' 'grigny'
 'saint

In [170]:
df24

,valeur_fonciere,code_postal,code_commune,nom_commune,type_local,surface_reelle_bati,nombre_pieces_principales,surface_terrain,longitude,latitude,score_espaces_verts,nb_ecoles,qualite,qualite_prix
140,211100.0,69004.0,69384,lyon 4e arrondissement,appartement,62.0,4.0,NaN,4.838431,45.778483,38.2,9.0,0.5,1.351477
141,211100.0,69004.0,69384,lyon 4e arrondissement,appartement,62.0,4.0,NaN,4.838431,45.778483,38.2,9.0,0.5,1.351477
142,211100.0,69004.0,69384,lyon 4e arrondissement,appartement,62.0,4.0,NaN,4.838431,45.778483,38.2,9.0,0.5,1.351477
143,211100.0,69004.0,69384,lyon 4e arrondissement,appartement,62.0,4.0,NaN,4.838431,45.778483,38.2,9.0,0.5,1.351477
144,211100.0,69004.0,69384,lyon 4e arrondissement,appartement,62.0,4.0,NaN,4.838431,45.778483,38.2,9.0,0.5,1.351477
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
473311,216000.0,69008.0,69388,lyon 8e arrondissement,appartement,104.0,5.0,NaN,4.883115,45.722048,40.2,19.0,0.5,1.320818
473312,216000.0,69008.0,69388,lyon 8e arrondissement,appartement,104.0,5.0,NaN,4.883115,45.722048,40.2,19.0,0.5,1.320818
473313,216000.0,69008.0,69388,lyon 8e arrondissement,appartement,104.0,5.0,NaN,4.883115,45.722048,40.2,19.0,0.5,1.320818
473314,216000.0,69008.0,69388,lyon 8e arrondissement,appartement,104.0,5.0,NaN,4.883115,45.722048,40.2,19.0,0.5,1.320818
